# Ещё больше Pandas

Мы загрузим датасет, на материале которого мы изучим некоторые более продвинутые методы Pandas:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data/countries.csv', decimal=",")
display(df.head())
df.info()

Обратите внимание, что в датасете есть пропуски. Мы должны будем иметь это в виду в дальнейшем, выполняя анализ.

Создайте сводную таблицу (`pivot_table`), которая содержит суммарные значения (`sum`) по населению (`Population`) и площади территории (`Area (sq. mi.)`) для каждого региона (`Region`).

In [ ]:
# место для кода
pd.pivot_table(df, values=['Population', 'Area (sq. mi.)'], index='Region', aggfunc=np.sum)

Воспользуйтесь методом `melt` для того, чтобы создать DataFrame, в котором вся информация собрана в трех столбцах, а именно
* `Country`,
* `Metric` (здесь будет имя переменной),
* `Measurement` (здесь будет значение этой переменной для данной страны).

Получившаяся таблица должна содержать 4540 строк.

In [ ]:
# место для кода
melted = pd.melt(df, id_vars='Country', var_name='Metric', value_name='Measurement')
melted.shape

In [ ]:
melted.head(10)

Выделите фрагмент из датафрейма, созданного выше при помощи метода melt, так, чтобы в нём содержались данные только по Афганистану (Afghanistan). Удалите столбец `Country` и задайте `Metric` в качестве индекса. 

*Подсказка: чтобы понять, как отфильтровать датафрейм, может быть полезным исследовать значения столбца `Country`.* `df['Country'].unique()`

In [ ]:
# место для кода
melted[melted['Country'] == 'Afghanistan ']

In [ ]:
afgh = melted[melted['Country'].str.contains('Afghanistan')]
afgh.drop('Country', axis=1, inplace=True)
afgh = afgh.set_index('Metric')
afgh

In [ ]:
#euro = df[df['Region'].str.contains('WESTERN EUROPE')]

Создайте новый столбец `Net inward migration`, содержащий булевские значения, обозначающие, в каких странах `net migration` имеет положительное значение, и создайте новый DataFrame, из которого были удалены те страны, в которых столбцы `Net migration`, `Birthrate` и `Deathrate` имеют отсутствующие значения.

Далее, создайте датафрейм с мульти-индексом из `Region` и `Net inward migration` и двумя столбцами: один содержащий среднее арифметическое значений из столбца `Birthrate` и второй -- среднее арифметическое по столбцу `Deathrate`. Используйте метод `.groupby`.

In [ ]:
# место для кода
df['Net inward migration'] = df['Net migration'] > 0
dfbd = df.dropna(subset = ['Net migration', 'Birthrate', 'Deathrate'])
gb = dfbd.groupby(['Region', 'Net inward migration'])['Birthrate', 'Deathrate'].mean()
gb

Подумайте о том, что именно мы вычислили, и что результат не обязательно отражает то, что мы ожидаем увидеть. Страны с небольшим населением влияют на средние значения в той же мере, что и страны с большим населением.

Мы хотим переформатировать результат (таблицу) таким образом, чтобы у таблицы был один индекс (`Region`), и она содержала средние значения по `Birthrate` и `Deathrate` только в тех случаях, где `Net inward migration` имеет значение `False`.

- Для этого воспользуйтесь методом `.unstack()`. 
- Затем удалите колонки с названием `True`, используйте для этого метод `.drop(columns=True, level=1)`
- Затем уберите 1-ый уровень индекса  с названиями столбцов и, таким образом, оставив только один уровень.  Используйте для этого метод `.columns.droplevel(1)`
- Отсортируйте результат (таблицу) по средним значениям `Birthrate` (в нисходящем порядке).
- В результате таблица содержит только колонки  `Birthrate` и `Deathrate`. 

In [ ]:
# место для кода
emig = gb.unstack().drop(columns=True, level=1)
emig.columns = emig.columns.droplevel(1)
emig = emig.sort_values(by='Birthrate', ascending=False)
emig

Используя метод `pd.cut`, создайте столбец с значениями 'Low', 'Medium' или 'High', которые показывают, в какую из категорий попадает та или иная страна. Выделите указанные категории на основании дохода, разбив диапазон численно. Сколько стран попадают в каждую из трех категорий?

In [ ]:
# место для кода
df['Income group (amount)'] = pd.cut(df['GDP ($ per capita)'], bins = 3, labels=['Low', 'Medium', 'High'])
df['Income group (amount)'].value_counts()

Создайте ещё один столбец с теми же значениями 'Low', 'Medium' и 'High', на этот раз используя метод `.qcut`, для того, чтобы получить одинаковое число стран в каждой из категорий.

In [ ]:
df['Income group (number)'] = pd.qcut(df['GDP ($ per capita)'], 3, labels=['Low', 'Medium', 'High'])
df['Income group (number)'].value_counts()

Далее, используя метод `.groupby`, покажите, сколько стран из каждого региона (`Region`) попадают в каждую из трех категорий согласно этому столбцу.

In [ ]:
# место для кода
reg_inc = df.groupby(['Region', 'Income group (number)']).size()
reg_inc
reg_inc.unstack()

Для каждого `Income group (amount)` покажите, сколько стран имеют и сколько не имеют `Net inward migration`. Используйте метод `pd.crosstab`.

In [ ]:
pd.crosstab(df['Income group (amount)'], df['Net inward migration'])

In [ ]:
#Correlations of columns
df.corr()